In [0]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import math

from keras.optimizers import RMSprop
from keras.layers import Dense, Input, Conv2D, Flatten, Reshape, Conv2DTranspose, BatchNormalization, LeakyReLU, Activation
from keras.models import Model
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.datasets import mnist
from keras.utils import plot_model
from keras import backend as K

Using TensorFlow backend.


In [0]:
def build_gen(inputs, image_size):
  
  image_resize = image_size //4
  kernel_size = 5
  layer_filter = [128, 64, 32, 1]
  
  x = Dense(image_resize * image_resize * 128)(inputs)
  x = Reshape((image_resize, image_resize, 128))(x)
  
  for filters in layer_filter:
    
    if filters > layer_filter[-2]:
      strides = 2
    else:
      strides = 1
      
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(filters = filters,
                        kernel_size = kernel_size,
                        strides = strides,
                        padding = 'same',
                        activation = 'sigmoid')(x)
  generator = Model(inputs, x, name = 'generator')
  return generator

In [0]:
def build_disc(inputs):
  
  kernel_size = 5
  layer_filter = [32, 64, 128, 256]
  
  x = inputs
  
  for filters in layer_filter:
    
    if filters == layer_filter[-1]:
      strides = 1
    else:
      strides = 2
      
    x = LeakyReLU(alpha = 0.2)(x)
    x = Conv2D(filters = filters,
               kernel_size = kernel_size,
               strides = strides,
               padding = 'same')(x)
  
  x = Flatten()(x)
  x = Dense(1, activation = 'sigmoid')(x)
  discriminator = Model(inputs, x, name = 'discriminator')
  return discriminator 

In [0]:
def build_and_train_models():
  (X_train, _), (_, _) = mnist.load_data()
  
  image_size = X_train.shape[1]
  X_train = np.reshape(X_train, [-1, image_size, image_size, 1])
  X_train = X_train.astype('float32')/255
  
  
  latent_size = 100
  batch_size = 64
  kernel_size = 5
  train_steps = 40000
  input_shape = (image_size, image_size, 1)
  
  inputs = Input(shape = input_shape, name = 'discriminator_input')
  discriminator = build_disc(inputs)
  discriminator.compile(loss = 'binary_crossentropy',
                        optimizer = RMSprop(lr = 2e-4, decay = 6e-8),
                        metrics = ['accuracy'])
  discriminator.summary()
  
  input_shape = (latent_size, )
  inputs = Input(shape = input_shape, name = 'z_input')
  generator = build_gen(inputs, image_size)
  generator.summary()
  
  optimizer = RMSprop(lr = 2e-4 * 0.5, decay = 6e-8 * 0.5)
  
  discriminator.trainable = False
  
  adversarial = Model(inputs,
                      discriminator(generator(inputs)),
                      name = "DCGAN_MNIST")
  adversarial.compile(loss = 'binary_crossentropy',
                      optimizer = optimizer,
                      metrics = ['accuracy'])
  adversarial.summary()
  
  models = (generator, discriminator, adversarial)
  
  params = (batch_size, latent_size, train_steps)
  train(models, X_train, params)

In [0]:
def train(models, X_train, params):
  generator, discriminator, adversarial = models
  
  batch_size, latent_size, train_steps = params
  
  save_interval = 500
  
  noise_input = np.random.uniform(-1.0, 1.0, size = [16, latent_size])
  
  train_size = X_train.shape[0]
  
  for i in range(train_steps):
    
    rand_indexes = np.random.randint(0, train_size, size = batch_size)
    
    real_images = X_train[rand_indexes]
    
    noise = np.random.uniform(-1.0, 1.0, size = [batch_size, latent_size])
    
    fake_images = generator.predict(noise)
    
    x = np.concatenate((real_images, fake_images))
    
    y = np.ones([2*batch_size, 1])
    y[batch_size:, :] = 0.0
    
    loss, acc = discriminator.train_on_batch(x,y)
    log = "%d: [discriminator loss: %f, acc: %f]" % (i, loss, acc)
    
    noise = np.random.uniform(-1.0, 1.0, size = [batch_size, latent_size])
   
    
    y = np.ones([batch_size, 1])
    
    loss, acc = adversarial.train_on_batch(noise, y)
    
    log = "%s [adversarial loss: %f, acc: %f]" % (log, loss, acc)
    print(log)
    
    if(i+1) % save_interval == 0:
      
      if(i + 1) == train_steps:
        show = True
      else:
        show = False
      
      
      plot_images(generator = generator, noise_input = noise_input,
                  show = show,
                  step = (i+1),
                  model_name = "DCGAN_MNIST")
      
      generator.save("dcgan_mnist.h5")
    

In [0]:
def plot_images(generator,
                noise_input,
                show=False,
                step=0,
                model_name="gan"):

    os.makedirs(model_name, exist_ok=True)
    filename = os.path.join(model_name, "%05d.png" % step)
    images = generator.predict(noise_input)
    plt.figure(figsize=(2.2, 2.2))
    num_images = images.shape[0]
    image_size = images.shape[1]
    rows = int(math.sqrt(noise_input.shape[0]))
    for i in range(num_images):
        plt.subplot(rows, rows, i + 1)
        image = np.reshape(images[i], [image_size, image_size])
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.savefig(filename)
    if show:
        plt.show()
    else:
        plt.close('all')


In [0]:
if __name__ == '__main__':
  build_and_train_models()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
discriminator_input (InputLa (None, 28, 28, 1)         0         
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        832       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          51264     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
conv

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0: [discriminator loss: 0.693507, acc: 0.351562] [adversarial loss: 0.766678, acc: 0.000000]
1: [discriminator loss: 0.628009, acc: 1.000000] [adversarial loss: 0.931216, acc: 0.000000]
2: [discriminator loss: 0.548598, acc: 0.992188] [adversarial loss: 0.978789, acc: 0.000000]
3: [discriminator loss: 0.435125, acc: 0.992188] [adversarial loss: 1.615110, acc: 0.000000]
4: [discriminator loss: 0.408429, acc: 0.898438] [adversarial loss: 1.004660, acc: 0.000000]
5: [discriminator loss: 0.312652, acc: 0.992188] [adversarial loss: 1.862697, acc: 0.000000]
6: [discriminator loss: 0.239250, acc: 0.976562] [adversarial loss: 1.243159, acc: 0.000000]
7: [discriminator loss: 0.191166, acc: 1.000000] [adversarial loss: 2.491068, acc: 0.000000]
8: [discriminator loss: 0.213998, acc: 0.937500] [adversarial loss: 1.166372, acc: 0.000000]
9: [discriminator loss: 0.155078, acc: 0.992188] [adversarial loss: 2.012482, acc: 0.000000]
10: [discriminator loss: 0.203350, acc: 0.929688] [adversarial loss: 0

In [0]:
from google.colab import files
files.download('dcgan_mnist.h5')